### Imports

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model


c:\Users\Max\Documents\Software\5930\finenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load the Dataset

In [2]:
# Load Alpaca dataset from Hugging Face or local JSON
dataset = load_dataset("tatsu-lab/alpaca")

# Preview the dataset structure
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 52002
    })
})


### Format, Scale, and Tokenize the Dataset

In [3]:

# Combine instruction, input, and output into a single text field
def format_example(example):
    instruction = example['instruction']
    input_text = example['input']
    output = example['output']
    if input_text:
        return f"Instruction: {instruction}\nInput: {input_text}\nOutput: {output}"
    else:
        return f"Instruction: {instruction}\nOutput: {output}"

formatted_dataset = dataset.map(lambda x: {"text": format_example(x)})

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-135M")

# Set the padding token to the <eos_token> (end of sentence token)
tokenizer.pad_token = tokenizer.eos_token

# If you want to use a new token for padding
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Tokenize dataset
def tokenize_function(examples):
    tokenized = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()  # Copy input_ids to labels
    return tokenized


tokenized_dataset = formatted_dataset.map(tokenize_function, batched=True)


In [4]:
print(tokenized_dataset)
print(tokenized_dataset["train"][0])


DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 52002
    })
})
{'instruction': 'Give three tips for staying healthy.', 'input': '', 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.', 'text': 'Instruction: Give three tips for staying healthy.\nOutput: 1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.', 'input_ids': [25464, 42, 13843, 1296, 5608, 327, 9286, 2458, 30, 198, 17597, 42, 216, 33, 30, 36693, 253, 8609, 2714, 284, 919, 2090, 288, 1453, 7568, 282, 5574, 284, 5136, 30, 3717, 34, 30, 15382, 5578, 288, 1446, 469, 1248, 3212, 284, 1837, 30, 3717, 35, 30, 5399,

In [5]:
# Scale datasets for testing
def scale_dataset(dataset, max_samples=1000):
    """Scale down a dataset to a maximum number of samples"""
    if len(dataset) > max_samples:
        scaled_indices = list(range(max_samples))
        return dataset.select(scaled_indices)
    return dataset

# Set your desired size
MAX_SAMPLES = 10000  # Adjust this number as needed

# Print original sizes
print(f"Original sizes - Train: {len(tokenized_dataset['train'])}")

# Scale the train dataset
tokenized_train = scale_dataset(tokenized_dataset["train"], MAX_SAMPLES)

# Optionally scale the eval dataset (if present or needed)
tokenized_eval = scale_dataset(
    tokenized_dataset["train"],  # Replace with tokenized_dataset["eval"] if you have a separate eval set
    max(50, int(MAX_SAMPLES * 0.1))  # Keep eval set ~10% of train, minimum 50 samples
)

# Print scaled sizes
print(f"Scaled sizes - Train: {len(tokenized_train)}, Eval: {len(tokenized_eval)}")


Original sizes - Train: 52002
Scaled sizes - Train: 10000, Eval: 1000


### Fine tuning setup

In [6]:
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from transformers import EarlyStoppingCallback

# Load the pretrained model
model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-135M")

# Configure LoRA
lora_config = LoraConfig(
    r=8,  # Rank for low-rank adaptation
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],  # Target query and value projections
    task_type="CAUSAL_LM"  # This is a causal language model
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Print trainable parameters
model.print_trainable_parameters()


trainable params: 460,800 || all params: 134,975,808 || trainable%: 0.3414


In [7]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",  # Evaluate periodically
    eval_steps=2000,              # Evaluate every 500 steps
    save_steps=2000,              # Save model every 500 steps
    logging_steps=100,          # Log progress every 1000 steps
    load_best_model_at_end=True,  # Load the best model after training
    metric_for_best_model="eval_loss",  # Use evaluation loss as the metric
    greater_is_better=False,     # Lower eval_loss is better
    learning_rate=5e-4,
    per_device_train_batch_size=4,
    num_train_epochs=10,
    save_total_limit=2,
    fp16=False,
)

c:\Users\Max\Documents\Software\5930\finenv\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### Train

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = tokenized_train,
    eval_dataset = tokenized_eval,  # Replace with validation set if available
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

trainer.train()


  0%|          | 101/25000 [00:14<59:39,  6.96it/s] 

{'loss': 1.3415, 'grad_norm': 0.14469361305236816, 'learning_rate': 0.000498, 'epoch': 0.04}


  1%|          | 201/25000 [00:28<57:34,  7.18it/s]  

{'loss': 0.3081, 'grad_norm': 0.12462545186281204, 'learning_rate': 0.000496, 'epoch': 0.08}


  1%|          | 301/25000 [00:42<57:40,  7.14it/s]  

...

100%|██████████| 25000/25000 [1:00:49<00:00,  7.22it/s]

{'loss': 0.2283, 'grad_norm': 0.1532580554485321, 'learning_rate': 0.0, 'epoch': 10.0}


100%|██████████| 25000/25000 [1:00:49<00:00,  6.85it/s]

{'train_runtime': 3649.2486, 'train_samples_per_second': 27.403, 'train_steps_per_second': 6.851, 'train_loss': 0.2538819474029541, 'epoch': 10.0}


TrainOutput(global_step=25000, training_loss=0.2538819474029541, metrics={'train_runtime': 3649.2486, 'train_samples_per_second': 27.403, 'train_steps_per_second': 6.851, 'total_flos': 3.27672594432e+16, 'train_loss': 0.2538819474029541, 'epoch': 10.0})